<a href="https://colab.research.google.com/github/GirishHiremath/DL/blob/main/Batch_norm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("fivethirtyeight")
%load_ext tensorboard

In [2]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

4422102/4422102 [==============================] - 0s 0us/step


In [3]:
tf.random.set_seed(42)
np.random.seed(42)

LAYERS = [ tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(100, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(10, activation="softmax")]


model = tf.keras.models.Sequential(LAYERS)

In [4]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trai

In [6]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid), verbose=2)

Epoch 1/10
1719/1719 - 7s - loss: 1.2819 - accuracy: 0.6229 - val_loss: 0.8886 - val_accuracy: 0.7160 - 7s/epoch - 4ms/step
Epoch 2/10
1719/1719 - 6s - loss: 0.7955 - accuracy: 0.7362 - val_loss: 0.7130 - val_accuracy: 0.7656 - 6s/epoch - 4ms/step
Epoch 3/10
1719/1719 - 6s - loss: 0.6816 - accuracy: 0.7721 - val_loss: 0.6427 - val_accuracy: 0.7900 - 6s/epoch - 4ms/step
Epoch 4/10
1719/1719 - 6s - loss: 0.6217 - accuracy: 0.7944 - val_loss: 0.5900 - val_accuracy: 0.8066 - 6s/epoch - 3ms/step
Epoch 5/10
1719/1719 - 6s - loss: 0.5832 - accuracy: 0.8074 - val_loss: 0.5582 - val_accuracy: 0.8202 - 6s/epoch - 3ms/step
Epoch 6/10
1719/1719 - 6s - loss: 0.5553 - accuracy: 0.8156 - val_loss: 0.5350 - val_accuracy: 0.8238 - 6s/epoch - 4ms/step
Epoch 7/10
1719/1719 - 6s - loss: 0.5338 - accuracy: 0.8224 - val_loss: 0.5157 - val_accuracy: 0.8304 - 6s/epoch - 3ms/step
Epoch 8/10
1719/1719 - 6s - loss: 0.5173 - accuracy: 0.8272 - val_loss: 0.5079 - val_accuracy: 0.8282 - 6s/epoch - 3ms/step
Epoch 9/

##Batch Norm approach One

In [7]:
del model

In [9]:
LAYER_BN = [
    tf.keras.layers.Flatten(input_shape=[28,28]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation='softmax')
]

model = tf.keras.models.Sequential(LAYER_BN)

In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization_2 (Batc  (None, 784)              3136      
 hNormalization)                                                 
                                                                 
 dense_4 (Dense)             (None, 300)               235500    
                                                                 
 batch_normalization_3 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_5 (Dense)             (None, 100)               30100     
                                                                 
 batch_normalization_4 (Batc  (None, 100)             

In [11]:
784 * 4 #mean, VAriance, gamma and Beta

3136

In [12]:
300 * 4

1200

In [13]:
100 * 4

400

In [14]:
3136 + 1200 + 400

4736

In [15]:
4736 / 2 # non trainable parameter

2368.0

In [16]:
266610 + 2368.0 # Trainable params

268978.0

In [17]:
266610 + 4736 # Total parameter

271346

In [18]:
bn1 = model.layers[1]

In [19]:
for variable in bn1.variables:
  print(variable.name, variable.trainable)

batch_normalization_2/gamma:0 True
batch_normalization_2/beta:0 True
batch_normalization_2/moving_mean:0 False
batch_normalization_2/moving_variance:0 False


In [20]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [21]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid), verbose=2)

Epoch 1/10
1719/1719 - 10s - loss: 0.8293 - accuracy: 0.7221 - val_loss: 0.5539 - val_accuracy: 0.8160 - 10s/epoch - 6ms/step
Epoch 2/10
1719/1719 - 9s - loss: 0.5703 - accuracy: 0.8035 - val_loss: 0.4792 - val_accuracy: 0.8378 - 9s/epoch - 5ms/step
Epoch 3/10
1719/1719 - 9s - loss: 0.5161 - accuracy: 0.8212 - val_loss: 0.4424 - val_accuracy: 0.8494 - 9s/epoch - 5ms/step
Epoch 4/10
1719/1719 - 9s - loss: 0.4789 - accuracy: 0.8315 - val_loss: 0.4211 - val_accuracy: 0.8566 - 9s/epoch - 5ms/step
Epoch 5/10
1719/1719 - 9s - loss: 0.4548 - accuracy: 0.8405 - val_loss: 0.4050 - val_accuracy: 0.8614 - 9s/epoch - 5ms/step
Epoch 6/10
1719/1719 - 9s - loss: 0.4387 - accuracy: 0.8446 - val_loss: 0.3930 - val_accuracy: 0.8640 - 9s/epoch - 5ms/step
Epoch 7/10
1719/1719 - 9s - loss: 0.4255 - accuracy: 0.8505 - val_loss: 0.3828 - val_accuracy: 0.8644 - 9s/epoch - 5ms/step
Epoch 8/10
1719/1719 - 9s - loss: 0.4123 - accuracy: 0.8540 - val_loss: 0.3758 - val_accuracy: 0.8670 - 9s/epoch - 5ms/step
Epoch 

##Batch Norm approach Two

In [22]:
del model

In [23]:
LAYERS_BN_BIAS_FALSE = [
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(300, use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dense(100, use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dense(10, activation="softmax")
]

model = tf.keras.models.Sequential(LAYERS_BN_BIAS_FALSE)

In [24]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [25]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 12s 6ms/step - loss: 1.0346 - accuracy: 0.6739 - val_loss: 0.6680 - val_accuracy: 0.7884
Epoch 2/10
1719/1719 [==============================] - 11s 6ms/step - loss: 0.6757 - accuracy: 0.7819 - val_loss: 0.5537 - val_accuracy: 0.8212
Epoch 3/10
1719/1719 [==============================] - 11s 6ms/step - loss: 0.5961 - accuracy: 0.8021 - val_loss: 0.4996 - val_accuracy: 0.8352
Epoch 4/10
1719/1719 [==============================] - 10s 6ms/step - loss: 0.5456 - accuracy: 0.8175 - val_loss: 0.4655 - val_accuracy: 0.8462
Epoch 5/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.5140 - accuracy: 0.8249 - val_loss: 0.4420 - val_accuracy: 0.8506
Epoch 6/10
1719/1719 [==============================] - 11s 6ms/step - loss: 0.4915 - accuracy: 0.8308 - val_loss: 0.4238 - val_accuracy: 0.8536
Epoch 7/10
1719/1719 [==============================] - 13s 7ms/step - loss: 0.4741 - accuracy: 0.8377 - val_loss: 0.4103 - val_ac